<a href="https://colab.research.google.com/github/yanncoadou/MLtutorials/blob/ESIPAP2023/BDT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><center>Machine learning hands-on</center></h1>
<h2><center>Decision tree algorithms</center></h2>

# Standard imports and useful functions

In [ ]:
# scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification, make_circles
from sklearn.metrics import plot_confusion_matrix, plot_roc_curve, accuracy_score, roc_auc_score, roc_curve, RocCurveDisplay

%matplotlib inline
import seaborn as sns # seaborn for nice plots
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
np.random.seed(31415) # set the np random seed for reproducibility

### Function to plot decision contours

In [ ]:
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

def my_plot_decision_regions(model, X, y, alpha=1.0, size=25, npts=10000, zoom=0.25, event5=False):
  x1min = X[:,0].min() - zoom
  x1max = X[:,0].max() + zoom

  x2min = X[:,1].min() - zoom
  x2max = X[:,1].max() + zoom
  
  x1 = np.random.uniform(x1min, x1max, npts)
  x2 = np.random.uniform(x2min, x2max, npts)

  if hasattr(model, "predict_proba"):
    z = model.predict_proba(np.vstack((x1,x2)).T)
  else:
    z = model.predict(np.vstack((x1,x2)).T)
  
  if len(z.shape) == 2:
    if z.shape[1] == 1:
      z = z.reshape(-1)
    elif z.shape[1] == 2:
      z = z[:,1].reshape(-1)

  fig, ax = plt.subplots()

  bottom = cm.get_cmap('Oranges', 128)
  top = cm.get_cmap('Blues_r', 128)

  newcolors = np.vstack((top(np.linspace(0, 1, 128+128)[-128:]),
                        bottom(np.linspace(0, 1, 128+128)[:128])))
  newcmp = ListedColormap(newcolors, name='OrangeBlue')


  ax.tricontour(x1, x2, z, levels=np.linspace(0.0-np.finfo(float).eps,1.0+np.finfo(float).eps,20,True), linewidths=0.1, colors='k', antialiased=True)
  cntr = ax.tricontourf(x1, x2, z, levels=np.linspace(0.0-np.finfo(float).eps,1.0+np.finfo(float).eps,20,True), cmap=newcmp)
  sctr0 = ax.scatter(X[y==0][:,0], X[y==0][:,1], alpha=alpha, s=size, c="#1f77b4", marker="s", edgecolors="k", linewidths=0.5)
  sctr1 = ax.scatter(X[y==1][:,0], X[y==1][:,1], alpha=alpha, s=size, c="#ff7f0e",  marker="^", edgecolors="k", linewidths=0.5)
  if event5: # showing particular swinger event
    sctr2 = ax.scatter(X[4][0], X[4][1], alpha=1, s=size*10, c="lightgreen",  marker="X", edgecolors="k", linewidths=1)
  fig.colorbar(cntr, ax=ax)
  # ax.set(xlim=(x1min, x1max), ylim=(x2min, x2max))

  plt.show()

### Function to plot ROC curves

In [ ]:
def my_roc_curves(models, X_test, y_test, weights_test=np.array([])):
  if weights_test.size == 0:
    weights_test = np.ones(len(X_test))
  for i, clf in enumerate(models):
    if hasattr(clf, "predict_proba"):
      y_preds_clf = clf.predict_proba(X_test)[:,1].reshape(-1)
    else:
      y_preds_clf = clf.predict(X_test)
    fpr_clf,tpr_clf,_ = roc_curve(y_true=y_test, y_score=y_preds_clf, sample_weight=weights_test)
    auc_test_clf = roc_auc_score(y_true=y_test, y_score=y_preds_clf, sample_weight=weights_test)
    plt.plot(fpr_clf, tpr_clf, label='{} (AUC  = {})'.format(clf.__class__.__name__,np.round(auc_test_clf,decimals=2)))
  plt.plot([0, 1], [0, 1], color='black', linestyle='--')
  plt.xlim([-0.05, 1.0])
  plt.ylim([0.0, 1.05])
  plt.xlabel('False Positive Rate')
  plt.ylabel('True Positive Rate')
  plt.title('Receiver Operating Characteristic')
  plt.legend(loc="lower right");

# Defining datasets

In [ ]:
# X = (x,y) coordinates; y = class
X1, y1 = make_circles(n_samples=1000, noise=0.1, factor=0.8)
X2, y2 = make_circles(n_samples=1000, noise=0.2, factor=0.2)
X = np.vstack((X1,X2/2))
y = np.hstack((y1,y2))

# Splitting in train and test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)
sns.scatterplot(x=X[:,0], y=X[:,1], hue=y);

# Classifier zoo

Play with various tree-based algorithms as implemented in scikit-learn.

In [ ]:
# Add classifiers into a list to access them later
classifiers = []

## Decision tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dtc = DecisionTreeClassifier()

In [ ]:
display(dtc.get_params())

In [ ]:
dtc.fit(X_train, y_train);
classifiers.append(dtc)

In [ ]:
from sklearn.tree import plot_tree
plt.figure(figsize=(15,10))
plot_tree(dtc)
plt.show();

---
How often is the prediction of the decision tree correct? Measured **on the testing or validation sample**, for instance with *accuracy*.

*Note*: MANY other measures of performance, see e.g. what is available in [scikit-learn](https://scikit-learn.org/stable/modules/model_evaluation.html).

In [ ]:
print("Accuracy:",accuracy_score(y_test, dtc.predict(X_test)))

---
Access to results:
- `predict` returns the class (0 or 1 if binary classifier)
- `predict_proba` returns the probability of each class (if available)



In [ ]:
print("Truth:   \n",y_test[:5])
print("predict: \n",dtc.predict(X_test[:5]))
print("predict_proba: \n",dtc.predict_proba(X_test[:5]))

In [ ]:
# Plotting decision contours
try:
  from mlxtend.plotting import plot_decision_regions
except ImportError as e:
  !pip install mlxtend
  from mlxtend.plotting import plot_decision_regions

In [ ]:
# practical but limited contour-plotting function
plot_decision_regions(X_test, y_test, dtc);

In [ ]:
# defined at top of notebook
# can use class (0 or 1) or class probability when available
my_plot_decision_regions(dtc, X_test, y_test)

---
Receiver operating characteristic curve (ROC curve) and area under the curve (AUC).

<center> <img style="display: block; margin-left: auto; margin-right: auto; width: 30%;" alt="ROCcurve" width="30%" src="https://raw.githubusercontent.com/yanncoadou/MLtutorials/main/ROCcurve.png" > </center>


In [ ]:
my_roc_curves([dtc], X_test, y_test)

## AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
#abc = AdaBoostClassifier()
abc = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=4),n_estimators=100)
display(abc.get_params())

In [ ]:
abc.fit(X_train, y_train);
classifiers.append(abc)

In [ ]:
print("Truth:   \n",y_test[:5])
print("predict: \n",abc.predict(X_test[:5]))
print("predict_proba: \n",abc.predict_proba(X_test[:5]))

In [ ]:
my_plot_decision_regions(abc, X_test, y_test)
my_roc_curves([abc], X_test, y_test)

## Gradient boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
gbc = GradientBoostingClassifier(n_estimators=400,verbose=1)
display(gbc.get_params())

In [ ]:
gbc.fit(X_train, y_train)
classifiers.append(gbc)

In [ ]:
print("Truth:   \n",y_test[:5])
print("predict: \n",gbc.predict(X_test[:5]))
print("predict_proba: \n",gbc.predict_proba(X_test[:5]))

In [ ]:
my_plot_decision_regions(gbc, X_test, y_test, event5=True)
my_roc_curves([gbc], X_test, y_test)

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rfc = RandomForestClassifier(n_estimators=400,verbose=1)
display(rfc.get_params())

In [ ]:
rfc.fit(X_train, y_train)
classifiers.append(rfc)

In [ ]:
print("Truth:   \n",y_test[:5])
print("predict: \n",rfc.predict(X_test[:5]))
print("predict_proba: \n",rfc.predict_proba(X_test[:5]))

In [ ]:
my_plot_decision_regions(rfc, X_test, y_test)
my_roc_curves([rfc], X_test, y_test)

## Comparison

In [ ]:
my_roc_curves(classifiers, X_test, y_test)

## Other classifiers
We have only seen tree-based classifiers from scikit-learn above. Tree-based algorithms tend to be fast and to work decently out-of-the-box.

There are many more types of classifiers:

1.   implemented in scikit-learn: see the [user's guide](https://scikit-learn.org/stable/user_guide.html) for A LOT of different algorithms
2.   in various other packages:
- for decision trees (more powerful than scikit-learn implementations): [XGBoost](https://xgboost.readthedocs.io/en/stable/), [LightGBM](https://lightgbm.readthedocs.io/en/latest/), [CatBoost](https://catboost.ai/) (see below)

- for neural networks: [TensorFlow](https://www.tensorflow.org/) (tomorrow's tutorial), [PyTorch](https://pytorch.org/)


### XGBoost

In [ ]:
# preinstalled version 0.9.0 20230127
!pip install xgboost --upgrade # install 1.7.3 20230127

In [ ]:
# keep only previous random forest training for comparison
classifiers = [rfc]

In [ ]:
from xgboost import XGBClassifier
# tree_method="hist" is 10 times faster, however less robust against awkwards features
#   (not a bad idea to double check without it)
# Can even try tree_method="gpu_hist" if proper GPU installation
xgb = XGBClassifier(tree_method="hist")

xgb.fit(X_train, y_train) # note that XGB 1.3.X requires positive weights
classifiers.append(xgb)

In [ ]:
print("Truth:   \n",y_test[:5])
print("predict: \n",xgb.predict(X_test[:5]))
print("predict_proba: \n",xgb.predict_proba(X_test[:5]))

In [ ]:
my_roc_curves(classifiers, X_test, y_test)

### LightGBM

In [ ]:
# preinstalled version 2.2.3 20230127
!pip install lightgbm --upgrade # install 3.3.5 20230127
import lightgbm as lgb

In [ ]:
gbm = lgb.LGBMClassifier()
gbm.fit(X_train, y_train);
classifiers.append(gbm)

In [ ]:
print("Truth:   \n",y_test[:5])
print("predict: \n",gbm.predict(X_test[:5]))
print("predict_proba: \n",gbm.predict_proba(X_test[:5]))

In [ ]:
my_roc_curves(classifiers, X_test, y_test)

### CatBoost

In [ ]:
# not preinstalled 20230127
!pip install catboost # install 1.1.1 20230127
import catboost

In [ ]:
cat = catboost.CatBoostClassifier()
cat.fit(X_train, y_train)
classifiers.append(cat)

In [ ]:
print("Truth:   \n",y_test[:5])
print("predict: \n",cat.predict(X_test[:5]))
print("predict_proba: \n",cat.predict_proba(X_test[:5]))

In [ ]:
my_roc_curves(classifiers, X_test, y_test)

# High energy physics application

## Input dataset

Data created from ATLAS Open Data by David Rousseau. See doc:

http://opendata.atlas.cern/release/2020/documentation/datasets/intro.html

### Downloading dataset

In [ ]:
try:
  import google.colab
  COLAB = True # if running in COLAB
  print("You are running on Colab.")
except:
  COLAB = False # if not running on COLAB
  print("You are NOT running on Colab, you need to fix the data file path below.")
import tensorflow as tf

In [ ]:
if COLAB:
  #### Reading files from Google Drive
  # Need a Google account to be identified
  #!pip install PyDrive
  import os
  from pydrive.auth import GoogleAuth
  from pydrive.drive import GoogleDrive
  from google.colab import auth
  from oauth2client.client import GoogleCredentials
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)

In [ ]:
import os
datapath=""
if not os.path.isfile("dataWW_d1_600k.csv.gz"):
  if COLAB:
    #attach dataset from google drive 
    download = drive.CreateFile({'id': '1nlXp7P-xq_jip4aPE0j0mnPhYnIOcBv4'})
    download.GetContentFile("dataWW_d1_600k.csv.gz")
    !ls -lrt
  else :
    # Make sure the file is available locally. 
    # Should be downloaded from https://drive.google.com/uc?id=1nlXp7P-xq_jip4aPE0j0mnPhYnIOcBv4
    !ls -lrt # what is in the local directory
    datapath="/directory/where/you/stored/dataWW_d1_600k.csv.gz"
    !ls -lrt {datapath} # what is in the data directory
    datapath=os.path.abspath(datapath).replace("\ ", " ")  # try to normalise the path (annoyance with the space)
    print ("Will take data from : ",datapath)
filename=os.path.join(datapath,"dataWW_d1_600k.csv.gz")
print("File to be loaded: ")
!ls -l {filename}

# Loading dataset
dfall = pd.read_csv(filename) 
print ("\nFile loaded with ",dfall.shape[0], " events ")



You should now see:

`File loaded with  600000  events`

### Checking the content

In [ ]:
#dumping list of features
dfall.columns

In [ ]:
#examining first few events
display(dfall.head())

In [ ]:
#examining feature distributions
dfall.describe()

***Event weights***

In [ ]:
label_nevents = (dfall[dfall.label==0].shape[0], dfall[dfall.label==1].shape[0] )
print("Number of events per class (B, S):",label_nevents)

label_weights = (dfall[dfall.label==0].mcWeight.sum(), dfall[dfall.label==1].mcWeight.sum() ) 
print("Total weight per class (B, S):    ",label_weights)

## Event selection

Only keep events with exactly two leptons for this exercise.

Only keep events with positive weight, as many ML tools choke on negative weight.

*Note: This is in principle WRONG, only possibly valid if your positive and negative weight events are statistically similar (could then also take the absolute value of the weight to increase statistics).*


In [ ]:
print ("Df shape before selection:", dfall.shape)

fulldata=dfall[ (dfall.lep_n==2) & (dfall.mcWeight > 0)]  

print ("Df shape after selection: ",fulldata.shape)

In [ ]:
# Hide label and weights in separate vectors (not discriminating features)
# WARNING : there should be neither selection nor shuffling later on! (otherwise misalignement)
target = fulldata["label"]
weights = fulldata["mcWeight"]

# for simplicity only keep some features
# this is actually making a deep copy from fulldata
data=pd.DataFrame(fulldata, columns=["met_et","met_phi","lep_pt_0","lep_pt_1",'lep_phi_0', 'lep_phi_1'])
#data=pd.DataFrame(fulldata, columns=["met_et","met_phi","lep_pt_0","lep_pt_1",'lep_eta_0', 'lep_eta_1', 'lep_phi_0', 'lep_phi_1','jet_n','jet_pt_0',
#       'jet_pt_1', 'jet_eta_0', 'jet_eta_1', 'jet_phi_0', 'jet_phi_1']

print ("Df shape of dataset to be used:",data.shape)

### Plotting variables

In [ ]:
fig,ax=plt.subplots(1, 2, figsize=(12, 5))
data['met_et'].plot.hist(title='Missing Transverse Energy', log=True, ax=ax[0])
data[data.lep_pt_0+data.lep_pt_1>1000]['met_et'].plot.hist(bins=np.linspace(0,400,50),title='Missing Transverse Energy for large lepton Pt', ax=ax[1]);

In [ ]:
ax=data[target==0].plot.scatter(x='met_et', y='lep_pt_0',color="b",label="B")
data[target==1].plot.scatter(x='met_et', y='lep_pt_0',color="r",label="S",alpha=.5,ax=ax);

In [ ]:
data[data.lep_pt_0+data.lep_pt_1>2000].head()

In [ ]:
ax=data[target==0].hist(weights=weights[target==0],figsize=(15,12),bins=50,color='b',alpha=0.5,density=True,label="B")
ax=ax.flatten()[:data.shape[1]] # to avoid error if holes in the grid of plots (like if 7 or 8 features)
data[target==1].hist(weights=weights[target==1],figsize=(15,12),bins=50,color='r',alpha=0.5,density=True,ax=ax,label="S");


## Sample splitting

In [ ]:
np.random.seed(31415) # set the random seed (used for the train/test splitting)

from sklearn.model_selection import train_test_split
train_size = 0.75 # fraction of sample used for training
val_size = 0.2 # fraction of training sample used for validation

# split only train/test
#X_train, X_test, y_train, y_test, weights_train, weights_test = \
#    train_test_split(data, target, weights, train_size=train_size)

#split in train/validation/test
X_holdout, X_test, y_holdout, y_test, weights_holdout, weights_test = \
    train_test_split(data, target, weights, train_size=train_size)
X_train, X_val, y_train, y_val, weights_train, weights_val = \
    train_test_split(X_holdout, y_holdout, weights_holdout, train_size=1-val_size)

print("Training sample:  ", X_train.shape)
print("Validation sample:", X_val.shape)
print("Testing sample:   ", X_test.shape)

In [ ]:
# Weight handling
class_weights_train = (weights_train[y_train == 0].sum(), weights_train[y_train == 1].sum())
print ("class_weights_train (B, S):",class_weights_train)

for i in range(len(class_weights_train)): # could have more than two classes
    weights_train[y_train == i] *= max(class_weights_train)/ class_weights_train[i] #equalize number of background and signal event
    weights_test[y_test == i] *= 1/(1-train_size) # increase test weight to compensate for sampling
    weights_val[y_val == i] *= 1/val_size/train_size # increase val weight to compensate for samplings
    
print ("Test:  total weight sig", weights_test[y_test == 1].sum())
print ("Test:  total weight bkg", weights_test[y_test == 0].sum())
print ("Train: total weight sig", weights_train[y_train == 1].sum())
print ("Train: total weight bkg", weights_train[y_train == 0].sum())
print ("Val:   total weight sig", weights_val[y_val == 1].sum())
print ("Val:   total weight bkg", weights_val[y_val == 0].sum())


## Boosted tree training

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score # for binary classification if x > 0.5 -> 1 else -> 0
xgb = XGBClassifier(tree_method="hist")
xgb.fit(X_train, y_train.values,
        sample_weight=weights_train.values) # note that XGB 1.3.X requires positive weight


In [ ]:
my_roc_curves([xgb], X_test, y_test, weights_test)

### Training monitoring

In [ ]:
eval_set = [(X_train, y_train), (X_val, y_val)]
weight_set = [weights_train, weights_val]

In [ ]:
# previous XGBoost instantiation:
#   xgb = XGBClassifier(tree_method="hist")
xgb = XGBClassifier(tree_method="hist",
                    eval_metric=["logloss","auc","error"])
# Note: could also have used set_params instead of passing eval_metric in constructor:
#       xgb = XGBClassifier(tree_method="hist")
#       xgb.set_params(eval_metric=["logloss","auc","error"])

In [ ]:
# previous training command line:
#  xgb.fit(X_train, y_train,
#          sample_weight=weights_train.values)
xgb.fit(X_train, y_train,
        sample_weight=weights_train.values,
        eval_set=eval_set,
        sample_weight_eval_set=weight_set);



In [ ]:
# retrieve performance metrics
results = xgb.evals_result()
epochs = len(results['validation_0']['error'])
x_axis = range(0, epochs)
# plot log loss
fig, ax = plt.subplots()
ax.plot(x_axis, results['validation_0']['logloss'], label='Train')
ax.plot(x_axis, results['validation_1']['logloss'], label='Validation')
ax.legend()
plt.ylabel('Log Loss')
plt.title('XGBoost Log Loss')
plt.show()
# plot classification error
fig, ax = plt.subplots()
ax.plot(x_axis, results['validation_0']['error'], label='Train')
ax.plot(x_axis, results['validation_1']['error'], label='Validation')
ax.legend()
plt.ylabel('Classification Error')
plt.title('XGBoost Classification Error')
plt.show()
# plot AUC
fig, ax = plt.subplots()
ax.plot(x_axis, results['validation_0']['auc'], label='Train')
ax.plot(x_axis, results['validation_1']['auc'], label='Validation')
ax.legend()
plt.ylabel('AUC')
plt.title('XGBoost Area under the curve (AUC)')
plt.show()

In [ ]:
## Adding early stopping condition (will use last entry in eval_metric to monitor early stopping
xgb.set_params(early_stopping_rounds=10)
xgb.fit(X_train, y_train,
        sample_weight=weights_train.values,
        eval_set=eval_set,
        sample_weight_eval_set=weight_set)

*Why did it stop so early, after 11 trees only?*

*Play with* `n_estimators` *and stopping conditions to find better classifier.*

In [ ]:
#xgb = XGBClassifier(tree_method="hist")
#xgb.fit("FIX ME")

###Learning curve
Compute the AUC by varying the number of training events. Validation set remains the same.

In [ ]:
train_sizes=[0.01,0.05,0.1,0.2,0.5,0.75,1]
ntrains=[]
val_aucs=[]
train_aucs=[]
times=[]

import time

xgb = XGBClassifier(tree_method="hist")

for train_size in train_sizes:
  ntrain=int(len(X_train)*train_size)
  print("Training with ",ntrain," events")
  ntrains+=[ntrain]
  starting_time = time.time()

  # train using the first ntrain event of the training dataset
  xgb.fit(X_train[:ntrain], y_train[:ntrain],sample_weight=weights_train[:ntrain].values)
  training_time = time.time( ) - starting_time
  times+=[training_time]

  # score on validation dataset (always the same)
  y_val_xgb=xgb.predict_proba(X_val)[:,1]
  auc_val_xgb = roc_auc_score(y_true=y_val, y_score=y_val_xgb)
  val_aucs+=[auc_val_xgb]

  # score on the train dataset 
  y_train_xgb=xgb.predict_proba(X_train[:ntrain])[:,1]
  auc_train_xgb = roc_auc_score(y_true=y_train[:ntrain], y_score=y_train_xgb)
  train_aucs+=[auc_train_xgb]

dflearning=pd.DataFrame({"Ntraining":ntrains,
                         "val_auc":val_aucs,
                         "train_auc":train_aucs,
                         "time":times})
display(dflearning)

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(12, 5))
ax[0].grid()
ax[0].plot('Ntraining','train_auc',"o-",data=dflearning,label="Train",color="r")
ax[0].plot(dflearning.Ntraining,dflearning.val_auc,"o-",label="Validation",color="b")
ax[0].set_xlabel("Training examples")
ax[0].set_ylabel("AUC")
ax[0].legend()
ax[1].grid()
ax[1].plot('Ntraining','time',"o-",data=dflearning)
ax[1].legend()
ax[1].set_xlabel("Training examples")
ax[1].set_ylabel("Fit time [s]");

Could also use `learning_curve` in sklearn.

*Notes*:
* it does not handle event weights
* it does not allow to control testing dataset size


In [ ]:
from sklearn.model_selection import learning_curve
train_sizes,train_scores,test_scores,fit_times,_ = learning_curve(
     XGBClassifier(tree_method="hist"),
     X_train,y_train,
     train_sizes=[0.01,0.05,0.1,0.2,0.5,0.75,1],                  
     scoring='roc_auc',cv=5,
     return_times=True)

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(12, 5))
ax[0].set_title('Learning curves')
ax[0].set_xlabel("Training examples")
ax[0].set_ylabel("AUC")
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)
fit_times_mean = np.mean(fit_times, axis=1)
fit_times_std = np.std(fit_times, axis=1)
ax[0].grid()
ax[0].fill_between(
        train_sizes,
        train_scores_mean - train_scores_std,
        train_scores_mean + train_scores_std,
        alpha=0.3,
        color="r",
)
ax[0].fill_between(
        train_sizes,
        test_scores_mean - test_scores_std,
        test_scores_mean + test_scores_std,
        alpha=0.3,
        color="b",
)
ax[0].plot(train_sizes, train_scores_mean, "o-", color="r", label="Train")
ax[0].plot(train_sizes, test_scores_mean, "o-", color="b", label="Validation")
ax[0].legend(loc="best");

# Plot fit time vs Ntraining
ax[1].grid()
ax[1].plot(train_sizes, fit_times_mean, "o-")
ax[1].fill_between(
        train_sizes,
        fit_times_mean - fit_times_std,
        fit_times_mean + fit_times_std,
        alpha=0.3,
)
ax[1].set_xlabel("Training examples")
ax[1].set_ylabel("Fit time [s]")
ax[1].set_title("Scalability of model");


### Model saving

In [ ]:
xgb.save_model("XGBoost.model")
!ls -al

Reload a trained model:

In [ ]:
print("Prediction from original model:")
display(xgb.predict_proba(X_test[:5]))

reloaded_model=XGBClassifier()
reloaded_model.load_model("XGBoost.model")
print("Prediction from reloaded model:")
display(reloaded_model.predict_proba(X_test[:5]))

try:
  np.testing.assert_allclose(
      xgb.predict_proba(X_test), reloaded_model.predict_proba(X_test)
  )
  print("Original and reloaded models are identical")
except AssertionError:
  print("Watch out: original and reloaded models are different")

## Physics performance

### Feature importance
Feature importance allows to display the importance of each feature without rerunnning the training. It is obtained from internal algorithm quantities, like cumulated decrease of impurity, *during training*. Magnitude is arbitrary. It can be used as a not very reliable indication of which features are the most discriminant *for this particular training*.

Very straightforward with decision trees.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
classifiers = []

In [ ]:
gbc = GradientBoostingClassifier(n_estimators=10,verbose=1)
gbc.fit(X_train, y_train, sample_weight=weights_train)
classifiers.append(gbc)

In [ ]:
plt.bar(data.columns.values, gbc.feature_importances_)
plt.xticks(rotation=45)
plt.title("Feature importance")
plt.show()

*What about a different tree classifier?*

In [ ]:
xgb = XGBClassifier(tree_method="hist")
xgb.fit(X_train, y_train.values, sample_weight=weights_train.values)
classifiers.append(xgb)


In [ ]:
import lightgbm as lgb
gbm = lgb.LGBMClassifier()
gbm.fit(X_train, y_train.values,sample_weight=weights_train.values)
classifiers.append(gbm)

In [ ]:
import catboost
cat = catboost.CatBoostClassifier()
cat.fit(X_train, y_train,sample_weight=weights_train.values, verbose=False);
classifiers.append(cat)

In [ ]:
fig,ax=plt.subplots(2, 2, figsize=(18, 10))
for i, clf in enumerate(classifiers, start=0):
  ax[i//2,i%2].bar(data.columns.values, clf.feature_importances_)
  #ax[i//2,i%2].tick_params(labelrotation=45)
  ax[i//2,i%2].set_title("{} feature importance".format(clf.__class__.__name__))

*What conclusions can you draw from these plots?*

### Permutation importance

A better way to show the importance of each feature is Permutation Importance, where each feature in turn is replaced by an instance of an other event (effectively switching it off by randomising).

Works on any classifier, not just DT-based. Can be estimated on any sample, not just training set.

However, report can be misleading in case of highly correlated variables.

Available in [Scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.inspection.permutation_importance.html).
   


In [ ]:
from sklearn.inspection import permutation_importance
importances = []
for clf in classifiers:
  result = permutation_importance(clf, X_test, y_test, n_repeats=1, random_state=42, n_jobs=2)
  importances.append(pd.Series(result.importances_mean, index=list(data.columns.values)))
  

In [ ]:
fig,ax=plt.subplots(2, 2, figsize=(18, 10))
for i, (clf,importance) in enumerate(zip(classifiers,importances), start=0):
  importance.plot.bar(ax = ax[i//2,i%2], subplots=True)
  ax[i//2,i%2].tick_params(labelrotation=0)
  ax[i//2,i%2].set_title("{} feature importance".format(clf.__class__.__name__))

Another implementation targetting HEP:

https://github.com/aghoshpub/permutationImportancePhysics 

In particular it allows to : 
   * use event weights
   * display directly the loss in whatever criterion (ROC auc, asimov significance) when the feature is switched off
   * display the feature importance for a specific subset (for example the most signal like)
   * it can even display which feature has the largest impact on systematics


In [ ]:
!pip install PermutationImportancePhysics
from permutationimportancephysics.PermutationImportance import PermulationImportance # note the delibrate typo PermuLation
#XGBoost
PI_xgb = PermulationImportance(model=xgb, X=X_test.values,y=y_test,weights=weights_test,\
                       n_iterations=1,usePredict_poba=True, scoreFunction="amsasimov", colNames=list(data.columns.values))
#PI_xgb.dislayResults()
plott_xgb = PI_xgb.plotBars()

#LightGBM    
PI_gbm = PermulationImportance(model=gbm, X=X_test.values,y=y_test,weights=weights_test,\
                         n_iterations=1,usePredict_poba=True, scoreFunction="amsasimov", colNames=list(data.columns.values))
#PI_gbm.dislayResults()
plott_gbm = PI_gbm.plotBars()

### Significance

Asimov significance (from [arXiv:1007.1727](https://arxiv.org/abs/1007.1727) eq. 97):

> AMS = $\sqrt{2\left((s+b)\ln\left(1+\frac{s}{b}\right) - s\right)} = \frac{s}{\sqrt{b}}\left(1+\mathcal{O}(s/b)\right)$

In [ ]:
from math import sqrt, log
def amsasimov(s,b):
  if b<=0 or s<=0:
      return 0
  try:
      return sqrt(2*((s+b)*log(1+float(s)/b)-s))
  except ValueError:
      print(1+float(s)/b)
      print (2*((s+b)*log(1+float(s)/b)-s))

In [ ]:
vamsasimovs = []
for clf in classifiers:
  y_pred_clf = clf.predict_proba(X_test)[:,1].reshape(-1)
  int_pred_test_sig_clf = [weights_test[(y_test ==1) & (y_pred_clf > th_cut)].sum() for th_cut in np.linspace(0,1,num=50)]
  int_pred_test_bkg_clf = [weights_test[(y_test ==0) & (y_pred_clf > th_cut)].sum() for th_cut in np.linspace(0,1,num=50)]

  vamsasimov_clf = [amsasimov(sumsig,sumbkg) for (sumsig,sumbkg) in zip(int_pred_test_sig_clf,int_pred_test_bkg_clf)]
  print("Z({}): {}".format(clf.__class__.__name__,np.round(max(vamsasimov_clf),decimals=3)))
  vamsasimovs.append(vamsasimov_clf)

In [ ]:
for i, (clf,vamsasimov_clf) in enumerate(zip(classifiers,vamsasimovs), start=0):
  plt.plot(np.linspace(0,1,num=50),vamsasimov_clf, label='AMS (Z_max({}) = {})'.format(clf.__class__.__name__,np.round(max(vamsasimov_clf),decimals=3)))
plt.title("Significance")
plt.xlabel("Threshold")
plt.ylabel("Significance")
plt.legend()
#plt.savefig("Significance.pdf")
plt.show()

### Hyperparameter optimisation
Can be done by hand, with [random search](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html) or [grid search](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html).

Also dedicated packages doing Gaussian process optimisation or 'tree of Parzen estimators' (TPE) (e.g. [hyperopt](https://github.com/hyperopt/hyperop) or [optuna](https://optuna.org/)).

In [ ]:
import scipy.stats as stats
from sklearn.model_selection import RandomizedSearchCV

# specify parameters and distributions to sample from
param_dist_XGB = {'n_estimators': stats.randint(10, 500), #default 100
                  'learning_rate': stats.uniform(0.01, 0.5)} #def 0.3 
                  #'max_depth': stats.randint(3, 12)} # default 6

# default CV is 5 fold, reduce to 2 for speed concern
# default n_iter is 10 sets of parameters, reduce to 5 for speed concern
gsearch = RandomizedSearchCV(estimator = XGBClassifier(tree_method="hist"), 
                             param_distributions = param_dist_XGB, 
                             scoring='roc_auc',n_iter=5,cv=2,verbose=2)
gsearch.fit(X_train,y_train, sample_weight=weights_train);

In [ ]:
print ("Best parameters: ",gsearch.best_params_)
print ("Best score (on train dataset CV): ",gsearch.best_score_)
# Best model directly accessible if refit=True (default)
y_pred_gs = gsearch.predict_proba(X_test)[:,1]
print("... corresponding score on test dataset: ",roc_auc_score(y_true=y_test, y_score=y_pred_gs))

dfsearch=pd.DataFrame.from_dict(gsearch.cv_results_)
display(dfsearch.head())

fig,ax=plt.subplots(1, 3, figsize=(15, 5))
dfsearch.plot("param_n_estimators","mean_test_score",yerr="std_test_score",linestyle = 'None',marker="o", ax=ax[0])
ax[0].scatter(gsearch.best_params_['n_estimators'],gsearch.best_score_,color='red',marker="*",s=100,zorder=5)
dfsearch.plot("param_learning_rate","mean_test_score",yerr="std_test_score",linestyle = 'None',marker="o", ax=ax[1])
ax[1].scatter(gsearch.best_params_['learning_rate'],gsearch.best_score_,color='red',marker="*",s=100,zorder=5)
#dfsearch.plot("param_max_depth","mean_test_score",yerr="std_test_score",linestyle = 'None',marker="o", ax=ax[2])
#ax[2].scatter(gsearch.best_params_['max_depth'],gsearch.best_score_,color='red',marker="*",s=100,zorder=5);

# Your turn
Try and optimise a classifier on this dataset. You can play with:
*   type of classifier
*   hyperparameters of classifier
*   input variables
*   figure of merit

Remember that some algorithms are (much) more time consuming than others, and therefore take more time to optimise.

You can optimise by hand, or using some of the tools presented above.


In [ ]:
#your code goes here